# Result Analysis

Wellcome to the result analysis note book. Please make sure to run the cells in order once, to ensure evrything is working properly.

In [ ]:
import math
#Regular expressions
import re
#Pandas for data analysis
import pandas as pd
#Matplotlib to plot graphics
import matplotlib.pyplot as plt
#Numpy for mathematical computations
import numpy as np

from collections import namedtuple

from ipywidgets import widgets
from ipywidgets import interact

from IPython.display import display

#Imports for file operations
from os import listdir
from os.path import isfile

from matplotlib import rc

rc('text', usetex=True)
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Times']})

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

rc('font', size=SMALL_SIZE)          # controls default text sizes
rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def loadFile(fileName):
    file = open(fileName, 'r')
    Entry = namedtuple('Entry', ["track", "domain", "problem", "method"])
    data = dict()
    
    linePattern = re.compile("./(([^/]*)/)?([^/-]*).*/([^/]*)/([^/]*)/result:?(.*)")
    validPattern = re.compile("Plan is VALID")
    outOfMemoryPattern = re.compile(".*OutOfMemoryException.*")
    timePattern = re.compile("(.*)=([^ ]*)( ([^ ]*) ([^ ]*))?")
    stepsPattern = re.compile("(([0-9]*):)|(STEP ([0-9]*):)")

    for line in file:
        m = linePattern.match(line)
        if (m):
            track = m.group(2) if m.group(2) != None else ''
            domain = m.group(3).lower() if m.group(3) != None else ''
            problem = m.group(4).lower() if m.group(4) != None else ''
            method = m.group(5)  if m.group(5) != None else ''
            entry = Entry(track=track,domain=domain, problem=problem, method=method)
            if (not entry in data):
                data[entry] = {"solvingTime":0.0, "completeTime":0.0 ,"valid":False, "OutOfMemory":False, "steps": -1}
            info = m.group(6)

            m = validPattern.match(info)
            if (m):
                data[entry]["valid"] = True
            m = outOfMemoryPattern.match(info)
            if (m):
                data[entry]["OutOfMemory"] = True

            m = timePattern.match(info)
            if (m):
                wallClockTime = float(m.group(2))
                if (m.group(4) == None):
                    time = wallClockTime
                else:
                    userTime = float(m.group(4)) + float(m.group(5))
                    time = min(wallClockTime,userTime)

                time = round(time)
                if (time < 1):
                    time = 1

                data[entry][m.group(1)] = time

            m = stepsPattern.match(info)
            if (m):
                if (m.group(2) != None):
                    data[entry]["steps"] = m.group(2)
                else:
                    data[entry]["steps"] = m.group(4)

            #print(m.group(1),m.group(2),m.group(3),m.group(4))

    df = pd.DataFrame(data, dtype=float)
    df = df.transpose()
    df.index.names = ["track", "domain", "problem", "method"]
    df = df.reset_index()

    for i, row in df.iterrows():
      if (row["completeTime"] > 300):
        df.set_value(i,'valid',False)

    # Add min data
    minData = pd.DataFrame(df.groupby(["domain", "problem"], as_index=False)["completeTime"].min())
    minData = minData.rename(columns={"completeTime":"minCompleteTime"})
    df = pd.merge(df, minData, on=["domain", "problem"])

    #cominisatps2sun_nopre_MpC_r0.5


    #df = df[(df["track"] != 'seq-opt') & (df["track"] != 'seq-sat')]

    methodPattern = re.compile("(.*)_([^_-]*)(-([^_-]*))?_([^_]*)")
    # Split method information:
    def matchFunction(c, x):
        m = methodPattern.match(x)
        if (m):

            result = m.group(c)
            if (c == 5):
                mapping = {'n':'0_n', 's':'1_s', 'r0.5':'2_r0.5'}
                result = mapping[result];
            if result == None:
                result = ''
            return result
        else:
            if (c == 2):
                return 'MpC'
            return ''

    df = df.assign(encoding = [matchFunction(4, method) for method in df["method"]],
            encoder = [matchFunction(2, method) for method in df["method"]],
            incremental = [matchFunction(5, method) for method in df["method"]],
            solver = [matchFunction(1, method) for method in df["method"]],       
        )
    
    print("done.")
    return df

# Load File

Select the benchmark files to load here.

In [ ]:
files = [f for f in listdir('./') if isfile(f)]
files.sort()

fileWidget = widgets.Dropdown(
    options=files,
    description='File:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
fileWidget

In [ ]:
df = loadFile(fileWidget.value)
validRuns = df[df["valid"] == True]

# Solved Instances

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    display(x)
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_rows')

# Solved Instances
removedDomains =[]
removedDomains = ["openstacks","transport","visitall"]

solvedInstances = df[~df["domain"].isin(removedDomains)].groupby(["domain", "encoder", "encoding", "incremental"])["valid"].sum().unstack([1,2,3])
solvedInstances = solvedInstances.fillna(0)

## Add total row:
solvedInstances.loc['Total']= solvedInstances.sum()
pd.set_option('float_format', '{:,.0f}'.format)
print_full(solvedInstances.transpose())
#print(solvedInstances.to_latex())
pd.reset_option('float_format')

# Cactusplot

In the first cell you can sellect which methods to show in the second cell. The third cell shows the plot from the paper.

In [ ]:
allMethods = validRuns["method"].unique()
toggleMethods = widgets.SelectMultiple(
    options=list(allMethods),
    description='Methods:',
    disabled=False,
    width='100%'
#    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     icon='check'
)
toggleMethods

In [ ]:
methods = toggleMethods.value

from matplotlib import cm
colorMap = cm.viridis

for method in methods:
    execTimes = validRuns[(validRuns["method"] == method)].sort_values("completeTime")["completeTime"].tolist()
    numberOfProblems = list(range(1, len(execTimes) + 1))
     
    info = validRuns[(validRuns["method"] == method)].head(1).to_dict('records')[0]
    
    label = info['encoder'] + " " + info['encoding']
    if ((info['encoder'] == 'MpC') & (info['encoding'] == 'a')):
        label = "$\\forall$"
    if ((info['encoder'] == 'MpC') & (info['encoding'] == 'e')):
        label = "$\\exists$"
    if ((info['encoder'] == 'Srt') & (info['encoding'] == 'e')):
        label = "$R^2 \\exists$"
    if ((info['encoder'] == 'Srt') & (info['encoding'] == 'r')):
        label = "REINF."
    
    color = 'black'
    if (info['incremental'] == '0_n'):
        label += ' not i.'
        color = colorMap(0.8)
    if (info['incremental'] == '1_s'):
        label += ' single i.'
        color = colorMap(0.5)
    if (info['incremental'] == '2_r0.5'):
        label += ' double i.'
        color = colorMap(0.2)
    
    if (info["encoder"] == 'MpC'):
        linestyle = "solid"
    if (info["encoder"] == 'Srt'):
        linestyle = "dashed"
    plt.plot(numberOfProblems, execTimes, label=label, color=color, ls = linestyle)

plt.xlabel('number of instances solved')
plt.ylabel('time limit in seconds')
ax = plt.gca()
ax.set_xlim([50,110])
lgd = plt.legend(loc='best')

#plt.savefig('../images/cactus-plot-agil.svg', format='svg', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
SMALL_SIZE = 9
MEDIUM_SIZE = 10
BIGGER_SIZE = 11

rc('font', size=SMALL_SIZE)          # controls default text sizes
rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

methods = (
    'MpC',
    'cominisatps2sun_nopre_MpC-e_n',
    'cominisatps2sun_nopre_MpC-e_r0.5',
)

from matplotlib import cm
colorMap = cm.viridis

plt.figure(figsize=(3.32,1.8))
for method in methods:
    execTimes = validRuns[(validRuns["method"] == method)].sort_values("completeTime")["completeTime"].tolist()
    numberOfProblems = list(range(1, len(execTimes) + 1))
     
    info = validRuns[(validRuns["method"] == method)].head(1).to_dict('records')[0]
    
    label = info['encoder'] + " " + info['encoding']
    if ((info['encoder'] == 'MpC') & (info['encoding'] == 'e') & (info['incremental'] == '2_r0.5')):
        label = "incremental $\exists$"
        linestyle = "solid"
        color = "#e66101"
        dashes=''
    elif ((info['encoder'] == 'MpC') & (info['encoding'] == 'e') & (info['incremental'] == '0_n')):
        label = "non incr. $\exists$"
        linestyle = "--"     
        dashes=(4, 2)
        color = "#fdb863"
    else:
        label = "madagascar"
        dashes=(4,2,2,2)
        linestyle = "-."
        color = "#5e3c99"
    

    plt.plot(numberOfProblems, execTimes, label=label, ls=linestyle, dashes=dashes, color=color)
    


plt.xlabel('number of instances solved')
plt.ylabel('time limit in seconds')
ax = plt.gca()
ax.set_xlim([50,110])
lgd = plt.legend(loc='best',handlelength=2.2)

plt.savefig('images/cactus-plot-show.svg', format='svg', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

# Scatterplot

These are the scatter plots from the paper, for custom plots see below.

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

rc('font', size=SMALL_SIZE)          # controls default text sizes
rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def myScatterPlot(methods, ttype, xLabel, yLabel, outputFile, legend = True):
    plt.figure(figsize=(4,4))
    
    points = 0
    oneone = 0
    for method in methods:
        methodA, methodB, label = (list(method) + [None])[:3]
        dataMethodA = df[(df["method"] == methodA)]
        dataMethodB = df[(df["method"] == methodB)]

        infoA = df[(df["method"] == methodA)].head(1).to_dict('records')[0]
        infoB = df[(df["method"] == methodB)].head(1).to_dict('records')[0]

        compare = pd.merge(dataMethodA, dataMethodB, on=["track", "domain", "problem"], suffixes=["_A", "_B"])
        
        compare = compare[(compare["valid_A"] == True) | (compare["valid_B"] == True)]
        
        for i, row in compare.iterrows():
          if (row['valid_A'] == False):
            compare.set_value(i,ttype+'_A',330)
          if (row['valid_B'] == False):
            compare.set_value(i,ttype+'_B',330)
            
        #remove one one:
        compare = compare[(compare[ttype+'_A'] != 1) | (compare[ttype+'_B'] != 1)]
        
        points += len(compare)

        if (label == None):
            label = infoA['encoder'] + " " + infoA['encoding']
            if ((infoA['encoder'] == 'MpC') & (infoA['encoding'] == 'a')):
                label = "$\\forall$"
            if ((infoA['encoder'] == 'MpC') & (infoA['encoding'] == 'e')):
                label = "$\\exists$"
            if ((infoA['encoder'] == 'Srt') & (infoA['encoding'] == 'e')):
                label = "$R^2 \\exists$"
            if ((infoA['encoder'] == 'Srt') & (infoA['encoding'] == 'r')):
                label = "REINF."            
        timesMethodA, timesMethodB = zip(*compare[[ttype+"_A", ttype+"_B"]].values.tolist())
        plt.plot(timesMethodA, timesMethodB, label=label, marker='o', linestyle='')

    plt.loglog()    
    ax = plt.gca()
    ax.set_xlim([0.8,400])
    ax.set_ylim([0.8,400])

    #text
    xpos = 10 ** (math.log(ax.get_xlim()[1],10)/2)
    text = '('+str(points) + ' points)'
    ax.text(xpos, ax.get_ylim()[1]*0.53, text, horizontalalignment='center')

    #change marker:
    import itertools
    markers = [
        ('o', '#d8b365'),
        ('^', '#5ab4ac'),
        ('*', 'black'),
        ('s', 'grey'),
              ]
    #markers = '<^>v'
    for l, marker in zip(ax.lines, itertools.cycle(markers)):
        l.set_marker(marker[0])
        l.set_color(marker[1])

    if (legend):
        plt.legend(loc='best',numpoints=1,handlelength=0.5)

    plt.xlabel(xLabel)
    plt.ylabel(yLabel, labelpad=1)
        
    #line
    ax.plot(ax.get_xlim(), ax.get_ylim(), ls="--", c=".5", marker='')
    ax.plot([300,300], [1,300], ls="--", c=".5", marker='')
    ax.plot([1,300], [300,300], ls="--", c=".5", marker='')

    plt.xticks([1,10,100,300], [1,10,100,300])
    plt.yticks([1,10,100,300], [1,10,100,300])
    
    plt.savefig(outputFile, format='svg', bbox_inches='tight')
    plt.show()
    
  
myScatterPlot(methods = [
        ('cominisatps2sun_nopre_MpC-a_n', 'cominisatps2sun_nopre_MpC-a_s'),
        ('cominisatps2sun_nopre_MpC-e_n', 'cominisatps2sun_nopre_MpC-e_s'),
        ('cominisatps2sun_nopre_Srt-e_n', 'cominisatps2sun_nopre_Srt-e_s'),
        ('cominisatps2sun_nopre_Srt-r_n', 'cominisatps2sun_nopre_Srt-r_s'),
    ],
    ttype = "solvingTime",
    xLabel='non incremental',
    yLabel='single ended incremental',
    outputFile = 'images/n_vs_s-plot.svg')

myScatterPlot(methods = [
        ('cominisatps2sun_nopre_MpC-a_n', 'cominisatps2sun_nopre_MpC-a_r0.5'),
        ('cominisatps2sun_nopre_MpC-e_n', 'cominisatps2sun_nopre_MpC-e_r0.5'),
        ('cominisatps2sun_nopre_Srt-e_n', 'cominisatps2sun_nopre_Srt-e_r0.5'),
        ('cominisatps2sun_nopre_Srt-r_n', 'cominisatps2sun_nopre_Srt-r_r0.5'),
    ],
    ttype = "solvingTime",
    xLabel='non incremental',
    yLabel='double ended incremental',
    outputFile = 'images/r05_vs_n-plot.svg')

myScatterPlot(methods = [
        ('cominisatps2sun_nopre_MpC-a_s', 'cominisatps2sun_nopre_MpC-a_r0.5'),
        ('cominisatps2sun_nopre_MpC-e_s', 'cominisatps2sun_nopre_MpC-e_r0.5'),
        ('cominisatps2sun_nopre_Srt-e_s', 'cominisatps2sun_nopre_Srt-e_r0.5'),
        ('cominisatps2sun_nopre_Srt-r_s', 'cominisatps2sun_nopre_Srt-r_r0.5'),
    ],
    ttype = "solvingTime",
    xLabel='single ended incremental',
    yLabel='double ended incremental',
    outputFile = 'images/s_vs_r05-plot.svg')

myScatterPlot(methods= [
        ('MpC', 'cominisatps2sun_nopre_MpC-e_n', 'not i. $\exists$'),
        ('MpC', 'cominisatps2sun_nopre_MpC-e_r0.5', 'double i. $\exists$')
    ], 
    ttype='completeTime',
    xLabel='madagascar',
    yLabel='approach see label',
    outputFile = 'images/mpc_vs_us-plot.svg')

# Custom Scatter Plots

For custom scatter plots select exactly two methods. The order tetermines which is method A and which is method B. The first selected will be methodA the second methodB

In [ ]:
allMethods = validRuns["method"].unique()
toggleMethodsc = widgets.SelectMultiple(
    options=list(allMethods),
    description='Methods:',
    disabled=False,
    width='100%'
#    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     icon='check'
)
toggleMethodsc

In [ ]:
myScatterPlot(methods= [
        toggleMethodsc.value
    ], 
    ttype='solvingTime',
    xLabel='methodA',
    yLabel='methodB',
    outputFile = '../images/mpc_vs_us-plot.svg')